# Import de bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Logging

In [ ]:
import logging
from datetime import datetime
import os, os.path

filename = f'/content/drive/MyDrive/Colab Notebooks/logs/{datetime.today().strftime("%Y-%m-%d")}.log'



logging.FileHandler(filename, mode="a", encoding=None, delay=False)

logging.basicConfig(filename=filename,  
                    level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s",
                    force=True)

# 1 - Funções

## Download

In [31]:
def download_mapbiomas(download, link, path):
    if download:
        try: 
            
            
            df = pd.read_excel(link, sheet_name='COBERTURA_COL7') 
            
            df.columns = df.columns.astype('str')
            columns = {'municipality':'municipio',
                       'variable':'ano', 
                       'value':'valor'}
            
            df = remover_espacos(df)
            df = df.loc[df['UF'] == 'PA']
            df = pd.melt(df, id_vars = df[['municipality','class_id', 'level_4']].columns, value_vars =df.iloc[:,11:].columns)

            
            
            df = name_class_cobertura(df)
            df = df.drop(columns=['level_4'], errors='ignore')
            df = df.rename(columns= columns)
            df = df.dropna()

            
            # df.columns = columns


            df.to_parquet(path, index=False) # Salvando dados em .CSV
            logging.info('Extração de dados do mapbiomas concluído')
            print('[INFO] Extração de dados do mapbiomas concluído')
        except:
            raise BrokenPipeError('Erro de extração dos dados do mapbiomas')

def download_divisao_regional(download, link, path):
    if download:
        try: 
            pd.read_excel(link, #Extraindo dados da fonte (excel)
                    sheet_name='Planilha1').to_parquet(path, index=False) # Salvando dados em .CSV

            logging.info('Downloads de dados de divisão regional concluído')
            print('[INFO] Downloads de dados de divisão regional concluído')
        except:
            raise BrokenPipeError('Erro de extração dos dados de divisão regional')


def download_data_SIDRA(download, verbose=False):
    
    if download:
        lavouras_temporarias = {"https://apisidra.ibge.gov.br/values/t/1612/n6/in%20n3%2015/p/all/v/214,109,216/c81/":[2688, 2691, 2692,2694,2696,2697,2702,2703,2706,2706,2709,2711,2713,2714,2715]}
        produto_das_sivilcultura ={'https://apisidra.ibge.gov.br/values/t/291/n6/in%20n3%2015/p/all/v/142/c194/':[33247,33250,33253,33256,33258]}
        produto_das_pecuaria = {"https://apisidra.ibge.gov.br/values/t/3939/n6/in%20n3%2015/p/all/v/105/c79/":[2670, 2675, 2672, 2681, 2677]}
        produto_das_lavouras_permanentes = {"https://apisidra.ibge.gov.br/values/t/1613/n6/in%20n3%2015/p/all/v/216,2313,214/c82/":[45981,2720,2721,2722,2723,31619,2725,2727,2731,2733,2734,2736,2738,90001,2743,2745,2747]}

        
        tabelas_sidra = {
                        '1612':lavouras_temporarias,
                        '291':produto_das_sivilcultura,
                        '3939':produto_das_pecuaria,
                        '1613':produto_das_lavouras_permanentes,
                        }
        for k, v in tabelas_sidra.items():
            logging.info(f'Arquivo será baixado: tabela {k} da base de dados do IBGE')
            if verbose:
                print(f'Arquivo será baixado: tabela {k} da base de dados do IBGE')
            a = []
            for k2, v2 in v.items():
                for i in v2:
                    link = k2 + f"{i}/h/n"
                    
                    try:
                        logging.info(f'tentando: {i}')
                        if verbose:
                            print(f'tentando: {i}')
                        df = pd.read_json(link)
                        logging.info(f'concluido: {i}')
                        if verbose:
                            print(f'concluido: {i}')
                        a.append(df)
                    except:
                        print(f'falha: {i}')
                        logging.error(f'falha: {link}')


            table = pd.concat(a)
            if verbose:
                print('tabelas concatenadas')
            if k == '1612':
                table['tipo_producao'] = 'Produto das lavouras permanentes'
                if verbose:
                    print('tipo de producao definido para produto das lavouras permanentes')
            elif k == '1613':
                table['tipo_producao'] = "Produto das lavouras temporárias"
                if verbose:
                    print('tipo de producao definido para produto das lavouras temporárias')
            elif k == '291':
                table['tipo_producao'] = 'Silvicultura'
                if verbose:
                    print('tipo de producao definido para Silvicultura')
            elif k == '3939':
                table['tipo_producao'] = 'Pecuária'
                if verbose:
                    print('tipo de producao definido para Pecuária')
            
            

            table_columns_names = ['cod_mun', 'mun', 'cod_metrica', 'metrica', 
                                    'valor', 'cod_cidade', 'cidade', 'cod_ano', 
                                    'ano', 'cod_variavel', 'variavel', 'cod_produto', 
                                    'produto', 'tipo_producao']
            if verbose:
                print('colunas renomeadas')
            table.columns = table_columns_names
            table_path = f'/content/drive/MyDrive/Dados/raw/sidra/{k}.parquet'
            table.to_parquet(table_path, index= False)
            if verbose:
                print(f'tabelas salva em: {table_path}')
    else:
        print('SIDRA download skipped')
    

## Modelagem e limpeza

In [ ]:
def remover_espacos(raw_data):
    df_obj = raw_data.select_dtypes(['object']) 
    raw_data[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    return raw_data

In [ ]:
def filtrar_classes_de_interesse(classes_de_interesse, raw_data):
    
    dict_classes = dict(zip(raw_data.level_4.unique(), raw_data.class_id.unique()))
    lista_filtrados = []
    
    for i in classes_de_interesse:
        lista_filtrados.append(raw_data.loc[(raw_data.class_id == dict_classes[i]) & (raw_data.UF == 'PA')])

    dados_filtrados = pd.concat(lista_filtrados)[['municipality', 'class_id','level_4', '1985', '1986',
                                                '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
                                                '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
                                                '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
                                                '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']].fillna(0)

    return dados_filtrados

In [ ]:
def carregar_arquivos_base(download, links, paths):
    try:
        cobertura_terra = pd.read_parquet(paths[0]) # Carregando dados de uso da terra
        micro_reg = pd.read_parquet(paths[1]).rename(columns={'nome_mun':'municipality', 'CD_GEOCODI':'cod_municipality'}) # Carregando dados de microregiões
        
    except:
        raise FileNotFoundError('Dados de cobertura da terra não encontrados')

    return cobertura_terra, micro_reg

In [ ]:
def merge_dados_cobertura(dados, micro_reg):
    dados_merge = dados.merge(micro_reg, how = 'inner', on='municipality') # Mesclagem de dados
    dados_merge = dados_merge.rename(columns = {'municipality':'municipio', 'class_id':'id_classe',
                                                'nome_rgint':'regiao_intermediaria', 'cod_rgint':'cod_regiao_intermediaria',
                                                'nome_rgi':'regiao_imediata',
                                                'cod_rgi':'cod_regiao_imediata',
                                                'level_4':'nome_classe'})
    return dados_merge

In [ ]:
def name_class_cobertura(cobertura_long):
    for index, row in cobertura_long.iterrows():

        if row['level_4'] == 'Forest Formation':
            cobertura_long.loc[index,'variavel'] = 'Área de cobertura'
            cobertura_long.loc[index,'produto'] = 'Florest'
            cobertura_long.loc[index,'tipo_producao'] = 'Área de cobertura de Floresta'
        elif row['level_4'] == 'Pasture':
            cobertura_long.loc[index,'variavel'] = 'Área de cobertura'
            cobertura_long.loc[index,'produto'] = 'Pasto'
            cobertura_long.loc[index,'tipo_producao'] = 'Pastagem'
        elif row['level_4'] == 'Soy Beans':
            cobertura_long.loc[index,'variavel'] = 'Área de cobertura'
            cobertura_long.loc[index,'produto'] = 'Soja (em grão)'
            cobertura_long.loc[index,'tipo_producao'] = 'Produto das lavouras temporárias'
        elif row['level_4'] == 'Other Temporary Crops':
            cobertura_long.loc[index,'variavel'] = 'Área de cobertura'
            cobertura_long.loc[index,'produto'] = 'Outros'
            cobertura_long.loc[index,'tipo_producao'] = 'Produto das lavouras temporárias'
        elif row['level_4'] == 'Other Perennial Crops':
            cobertura_long.loc[index,'variavel'] = 'Área de cobertura'
            cobertura_long.loc[index,'produto'] = 'Outros'
            cobertura_long.loc[index,'tipo_producao'] = 'Produto das lavouras permanentes'
        elif row['level_4'] == 'Forest Plantation':
            cobertura_long.loc[index,'variavel'] = 'Área de cobertura'
            cobertura_long.loc[index,'produto'] = 'Silvicultura'
            cobertura_long.loc[index,'tipo_producao'] = 'Silvicultura'

    cobertura_long['metrica'] = 'Hectares'
    return cobertura_long

Pendente correção total

In [ ]:
# União das tabelas da SIDRA
def load_sidra_unified():
    lavouras_temporarias = pd.read_parquet(f'/content/drive/MyDrive/Dados/raw/sidra/1612.parquet') # Table of temporary crops
    lavouras_permanentes = pd.read_parquet(f'/content/drive/MyDrive/Dados/raw/sidra/1613.parquet') # Table of permanents crops
    sivilcultura = pd.read_parquet(f'/content/drive/MyDrive/Dados/raw/sidra/291.parquet')                  # Table of florestry
    pecuaria = pd.read_parquet(f'/content/drive/MyDrive/Dados/raw/sidra/3939.parquet')                         # Livestocks

    micro_regioes = pd.read_parquet('/content/drive/MyDrive/Dados/raw/geo/Microregioes.parquet')

    # Changing column name for cities code (ID)
    micro_regioes = micro_regioes.rename(columns = {'CD_GEOCODI':'cod_cidade'})

    iter_tables = [pecuaria, sivilcultura, lavouras_temporarias, lavouras_permanentes]

        # Unifying tables
    sidra_df = pd.concat(iter_tables)

    # Reseting index 
    sidra_df = sidra_df.reset_index(drop = True)

    # Formating cities names (Removing suffix) 
    sidra_df.cidade = sidra_df.cidade.str.replace(' - PA', '')

    # Strip on cities names
    sidra_df['cidade'] = sidra_df.cidade.str.strip()
   

    # Merging tables of the unyfied table and micro regions table
    sidra_df = sidra_df.merge(micro_regioes, how = 'inner', on = 'cod_cidade')

    # Changing the order of final table with the purpose of formating
    sidra_df = sidra_df[['cidade', 'cod_cidade', 'nome_rgi', 'cod_rgi', 'cod_rgint', 'nome_rgint', 
                        'tipo_producao', 'cod_variavel', 'variavel', 'cod_produto', 'produto',
                        'cod_metrica','metrica', 'valor', 'ano']]
    sidra_df = sidra_df.drop(columns = ['cod_produto', 'cod_metrica'])  
    sidra_df.columns = ['municipio', 'cod_municipality', 'regiao_intermediaria',
                        'cod_regiao_intermediaria', 'cod_regiao_imediata', 'regiao_imediata',
                        'tipo_producao', 'cod_variavel', 'variavel', 'produto','metrica', 'valor', 'ano']

    return sidra_df

In [15]:
def save_unified(df_cobertura, df_sifra):

    geo_df = pd.read_excel('/content/drive/MyDrive/Dados/raw/geo/Regioes_Geograficas_IBGE_Composicao_por_municipios.xls', sheet_name='PA').fillna(method='ffill')
    geo_df.columns = ['cod_mun', 'uf', 'municipio', 'reg_inter', 'nome_mun_reg_inter','reg_imed', 'nome_mun_reg_imed']
    geo_df = geo_df.drop(columns = ['nome_mun_reg_imed', 'nome_mun_reg_inter'])


    unified_df = pd.concat([df_sifra, df_cobertura])
    unified_df = unified_df[['municipio', 'tipo_producao', 'variavel', 'produto', 'metrica',
                            'ano', 'valor']]
    unified_df = unified_df.merge(geo_df, on='municipio',how='inner')
    unified_df['valor'] = pd.to_numeric(unified_df['valor'], downcast='integer', errors='coerce')
    unified_df['ano'] = unified_df['ano'].astype('int64')
    unified_df = unified_df.loc[(unified_df['ano'] >=1985)]
    unified_df.to_parquet('/content/drive/MyDrive/Dados/refined/table_full.parquet', index=False)

    return unified_df


# 2 - PIPELINE

In [32]:
# =================================================================
# ==================== Definição de parâmetros ====================
# =================================================================


# *-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
# ----------------------- Classe de interesse ---------------------
classes_de_interesse = ['Forest Formation', 'Pasture', 
                        'Soy Beans', 'Other Temporary Crops',
                        'Forest Plantation', 'Other Perennial Crops']

# ------------------------------ Links ----------------------------

link_mapbiomas = 'https://storage.googleapis.com/mapbiomas-public/brasil/downloads/1-tabela-geral-col7-mapbiomas-biomas-municipio-final.xlsx'
link_divisao_regional = 'https://geoftp.ibge.gov.br/organizacao_do_territorio/divisao_regional/divisao_regional_do_brasil/divisao_regional_do_brasil_em_regioes_geograficas_2017/tabelas/regioes_geograficas_composicao_por_municipios_2017_20180911.xlsx'

links = [link_mapbiomas, link_divisao_regional]


# ---------------------------- Caminhos ----------------------------

path_cobertura_terra = "/content/drive/MyDrive/Dados/raw/mapbiomas/cobertura.parquet"
path_microreg = '/content/drive/MyDrive/Dados/raw/geo/Microregioes.parquet'

paths = [path_cobertura_terra, path_microreg]
# *-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-




# =================================================================
# ======================= Download arquivos  ======================
# =================================================================

download_mapbiomas(True, link_mapbiomas, path_cobertura_terra)
download_divisao_regional(True, link_divisao_regional, path_microreg)

# =================================================================
# ======================= Load arquivos base ======================
# =================================================================

cobertura_terra, micro_reg = carregar_arquivos_base(True, links, paths)



# =================================================================
# ============================ PIPELINE ===========================
# =================================================================

# ------------------------------ SIDRA ------------------------------
download_data_SIDRA(False, True)
tabela_sidra = load_sidra_unified()

save_unified(cobertura_terra, tabela_sidra)

[INFO] Extração de dados do mapbiomas concluído
[INFO] Downloads de dados de divisão regional concluído
SIDRA download skipped


,municipio,tipo_producao,variavel,produto,metrica,ano,valor,cod_mun,uf,reg_inter,reg_imed
11,Abaetetuba,Pecuária,Efetivo dos rebanhos,Bovino,Cabeças,1985,1428.000000,1500107,PA,Belém,Abaetetuba
12,Abaetetuba,Pecuária,Efetivo dos rebanhos,Bovino,Cabeças,1986,3420.000000,1500107,PA,Belém,Abaetetuba
13,Abaetetuba,Pecuária,Efetivo dos rebanhos,Bovino,Cabeças,1987,3633.000000,1500107,PA,Belém,Abaetetuba
14,Abaetetuba,Pecuária,Efetivo dos rebanhos,Bovino,Cabeças,1988,4864.000000,1500107,PA,Belém,Abaetetuba
15,Abaetetuba,Pecuária,Efetivo dos rebanhos,Bovino,Cabeças,1989,5107.000000,1500107,PA,Belém,Abaetetuba
...,...,...,...,...,...,...,...,...,...,...,...
723468,Xinguara,Produto das lavouras temporárias,Área de cobertura,Outros,Hectares,2020,3912.590073,1508407,PA,Redenção,Xinguara
723469,Xinguara,Área de cobertura de Floresta,Área de cobertura,Florest,Hectares,2021,67297.304704,1508407,PA,Redenção,Xinguara
723470,Xinguara,Pastagem,Área de cobertura,Pasto,Hectares,2021,300973.161862,1508407,PA,Redenção,Xinguara
723471,Xinguara,Produto das lavouras temporárias,Área de cobertura,Soja (em grão),Hectares,2021,1232.578321,1508407,PA,Redenção,Xinguara
